## Objective
Determine how to use parallel processing (or multithreading) with sklearn GMM

In [1]:
%env MKL_NUM_THREADS=1
%env NUMEXPR_NUM_THREADS=1
%env OMP_NUM_THREADS=1

env: MKL_NUM_THREADS=1
env: NUMEXPR_NUM_THREADS=1
env: OMP_NUM_THREADS=1


In [1]:
from multiprocessing import Pool

import numpy as np
from sklearn.mixture import GaussianMixture

from tqdm import tqdm

In [2]:
import pickle
from joblib import Parallel, delayed
from copy import deepcopy
from pathlib import Path

# region Testing joblib
# def test(_):
#     data = np.random.random(1000).reshape((-1, 2))
#     gmm = GaussianMixture(n_components=6, init_params="kmeans", covariance_type="diag")
#     gmm.fit(data)
#     return None
# Parallel(n_jobs=20)(delayed(test)(x) for x in tqdm(range(300)))
# endregion

def job(test):
    from threadpoolctl import threadpool_limits
    with threadpool_limits(limits=1, user_api='blas'):
        for _ in range(10):
            data = np.random.random(100).reshape((-1, 2))
            gmm = GaussianMixture(n_components=6, init_params="kmeans", covariance_type="diag")
            gmm.fit(data)

with Pool(processes=12) as pool:
    tasks = range(100)
    for _ in tqdm(pool.imap_unordered(job, tasks), total=len(tasks)):
        pass

  0%|          | 0/100 [00:00<?, ?it/s]

 41%|████      | 41/100 [00:30<00:44,  1.32it/s]


KeyboardInterrupt: 

In [4]:
from threadpoolctl import threadpool_info
from pprint import pprint
import numpy
pprint(threadpool_info())

[{'architecture': 'SkylakeX',
  'filepath': '/home/mea/anaconda3/envs/prop/lib/libopenblasp-r0.3.20.so',
  'internal_api': 'openblas',
  'num_threads': 1,
  'prefix': 'libopenblas',
  'threading_layer': 'pthreads',
  'user_api': 'blas',
  'version': '0.3.20'},
 {'filepath': '/home/mea/anaconda3/envs/prop/lib/python3.7/site-packages/scikit_learn.libs/libgomp-a34b3233.so.1.0.0',
  'internal_api': 'openmp',
  'num_threads': 1,
  'prefix': 'libgomp',
  'user_api': 'openmp',
  'version': None}]
